# Introduction
State notebook purpose here

### Get source folder and append to sys directory

In [1]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)
# Data path example
#pump_data_path = os.path.join(PROJ_ROOT,
#                              "data",
#                              "raw",
#                              "pumps_train_values.csv")

/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program


### Imports
Import libraries and write settings here.

In [2]:
# Data manipulation
import matplotlib.pyplot as plt
from IPython import get_ipython
from IPython.core.interactiveshell import InteractiveShell
import pandas as pd
import numpy as np
import feedparser
from google.cloud import storage
from io import StringIO

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.join(PROJ_ROOT, 'flask_app',
                                                            "flask-app-test-317210-0c49e7d7d9cb.json")

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
InteractiveShell.ast_node_interactivity = 'all'
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 1
# Use %aimport module to reload each module

# Visualizations

# Analysis/Modeling
Do work here

In [3]:
def get_new_data():
    feed = "http://dosairnowdata.org/dos/RSS/HoChiMinhCity/HoChiMinhCity-PM2.5.xml"
    NewsFeed = feedparser.parse(feed)
    train = pd.DataFrame.from_dict(NewsFeed, orient='index')
    train2 = pd.DataFrame.from_dict(train.loc['entries', :].values[0])
    train2 = train2[['title', 'aqi']]
    train2.rename(columns={'title': 'time', 'aqi': 'AQI_h'}, inplace=True)
    train2 = train2.astype({'time': 'datetime64[ns]', 'AQI_h': 'float'})
    train2['site_id'] = 49
    train2['time']
    train2.set_index(['site_id', 'time'], inplace=True)
    train2['AQI_h_label'] = categorize_AQI(train2['AQI_h'])
    train2['AQI_h_I'] = train2['AQI_h_label'].cat.codes + 1
    train2['Continous length'] = 0
    return train2

def get_past_data_from_bucket_as_dataframe():
    """Read a blob"""
    bucket_name = "deep_learning_model_bucket"
    blob_name = "past_data.csv"
    
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    return_data = blob.download_as_text()
    return_data = StringIO(return_data)
    df = pd.read_csv(return_data, sep=",", header=0, index_col=False)
    df = df.astype({'time': 'datetime64[ns]', 'AQI_h': 'float'})
    df.set_index(['site_id', 'time'], inplace=True)
    return df
def concat_past_and_new_data():
    idx = pd.IndexSlice
    past_data = get_past_data_from_bucket_as_dataframe()
    new_data = get_new_data()
    max_past = past_data.index.get_level_values(1).max() + pd.Timedelta(hours=1)
    max_new = data_df.index.get_level_values(1).max()
    past_data = pd.concat([past_data, data_df.loc[idx[:, max_past:max_new], :]])
    return past_data

def delete_past_data_from_bucket():
    bucket_name = "deep_learning_model_bucket"
    blob_name = "past_data.csv"
    
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    return_data = blob.delete()
    print(return_data)
    return "Deleted"
def create_new_file_in_bucket():
    bucket_name = "deep_learning_model_bucket"
    blob_name = "past_data.csv"
    
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    return_data = blob.upload_from_filename('test.csv')
    print(return_data)
    return "Created"
def categorize_AQI(AQI_data):
    """
    Input: Series of AQI_values
    Output: Series of AQI category
    7 categories [Good, Moderate, Unhealthy for Sensitive, Unhealthy, Very Unhealthy, Hazardous, Out of AQI]
    range of categories [0-50, 51-100, 101-150, 151-200, 201-300, 301-500, >500]
    """
    bins = [-1, 50, 100, 150, 200, 300, 500, np.inf]
    labels = ["Good", "Moderate", "Unhealthy for Sensitive",
              "Unhealthy", "Very Unhealthy", "Hazardous", "Beyond AQI"]
    return pd.cut(AQI_data, bins=bins, labels=labels)

In [4]:
# get data from newsfeed
# read data from old file
# delete old file
# record new file with new data with same name
# Ah, I get it, I need to make a HTTP call every hour in app engine

In [6]:
data_df = get_new_data()
past_data = get_past_data_from_bucket_as_dataframe()

In [ ]:
# Get max time for past data
# Get max time for new data
# Get time from max_past to max_new
# Update dataframe
# Delete old data
# Update file

In [12]:
new_data = get_data_for_prediction()
past_data = get_past_data_from_bucket_as_dataframe()
new_data.to_csv('test.csv')
create_new_file_in_bucket()

None


'Created'

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here